In [1]:
import pandas as pd
#import full sentence
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',None)

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/talhaanwarch/Offensive-Language-Detection/master/Data/olid-training-v1.0.tsv',sep='\t')
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans what their take on this is.,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,NOT,NaN,NaN


In [3]:
df=df[['tweet','subtask_a']]
df.columns=['tweet','label']

# **Split the Data**

In [4]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(df,test_size=0.2)

#**Preprocess the Data**

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer= WordNetLemmatizer()
w_tokenizer=nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
import string
def  preprocess(df):
  df["tweet"]=df["tweet"].str.lower()
  df['tweet']=df["tweet"].str.replace('\d+','')#remove digits
  #remove @user
  df['tweet']=df['tweet'].str.replace('@user','')
  #remove url
  df["tweet"]=df["tweet"].str.replace('https[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','')

  #remove punctuations
  df["tweet"]=df["tweet"].apply(lambda x:x.translate(str.maketrans('','',string.punctuation)))

  df["tweet"]=df["tweet"].str.replace('[^\w\s]','')

  #lemmatize
  df["tweet"]=df["tweet"].apply(lambda x: [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
  #remove Stop words
  df["tweet"]=df["tweet"].apply(lambda x: [item for item in x if item not in stop ])
  #convert list to string
  df["tweet"]=df["tweet"].apply(lambda x : " ".join(x))
  return df

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\('
<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\('
<>:14: SyntaxWarning: invalid escape sequence '\w'
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\187636233.py:4: SyntaxWarning: invalid escape sequence '\d'
  df['tweet']=df["tweet"].str.replace('\d+','')#remove digits
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\187636233.py:9: SyntaxWarning: invalid escape sequence '\('
  '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','')
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\187636233.py:14: SyntaxWarning: invalid escape sequence '\w'
  df["tweet"]=df["tweet"].str.replace('[^\w\s]','')


In [7]:
train_df=preprocess(train_df)
test_df=preprocess(test_df)

train_df.head()

,tweet,label
5722,time,NOT
2623,yep issomebody going may finally exposed,NOT
12922,gtgtgtgtgt funner fact company wa created gtgtgtcialtltlt world know listen please url,NOT
6099,corrupt liberal government steal people liberal work,OFF
3032,superman ha morally superior matter excluding maybe love interest 😂 pure good nice yet still superior didnt see 🤷🏼‍♀️ disappointed 😭,NOT


In [8]:
from sklearn.preprocessing import LabelEncoder
enocder=LabelEncoder()
train_df['label']=enocder.fit_transform(train_df['label'])
test_df['label']=enocder.transform(test_df['label'])

In [9]:
train_df.head()

,tweet,label
5722,time,0
2623,yep issomebody going may finally exposed,0
12922,gtgtgtgtgt funner fact company wa created gtgtgtcialtltlt world know listen please url,0
6099,corrupt liberal government steal people liberal work,1
3032,superman ha morally superior matter excluding maybe love interest 😂 pure good nice yet still superior didnt see 🤷🏼‍♀️ disappointed 😭,0


In [10]:
train_df.label.value_counts()

label
0    7080
1    3512
Name: count, dtype: int64

#**Model Training**

create class weights

In [11]:
train_df.label

5722     0
2623     0
12922    0
6099     1
3032     0
2921     0
6341     1
9478     0
6699     1
6156     0
8248     1
387      0
1677     0
4799     0
10307    0
2255     0
12433    1
7545     0
9519     1
7901     0
5242     0
9723     1
3880     0
2219     1
1833     1
3232     1
1985     0
1137     0
12987    0
10342    0
11042    1
12542    0
6708     1
4851     0
94       1
329      0
3929     1
2338     1
13028    1
7551     0
12653    0
5565     0
7836     1
11784    1
5664     0
9937     1
1997     1
9294     0
6019     1
10075    0
1975     0
8499     1
7675     1
4861     0
2349     1
12825    0
3665     0
6603     1
13201    0
48       0
7886     0
8785     1
3062     0
11299    1
13173    1
7660     0
6983     0
12938    0
3711     0
11298    0
8789     1
4080     0
734      0
5495     0
11104    1
11346    0
11917    0
10811    1
1422     0
7538     0
13218    0
11064    0
2648     1
895      1
12213    0
3677     0
6671     1
11681    1
2883     0
10873    0
12066    0

In [12]:
from sklearn.utils import class_weight
import numpy as np
class_weights= class_weight.compute_class_weight('balanced',classes=np.unique(train_df.label),y=train_df.label.to_numpy())
class_weights=dict(enumerate(class_weights))
class_weights

{0: np.float64(0.7480225988700565), 1: np.float64(1.5079726651480638)}

#**Hyper-Parameter tuning**

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer

def logistic_param_selection(train_df):
  C=[1,3,5,7,9,10,12,15,20,25]
  #solver=['newton-cg','lbfgs','liblinear','sag','saga']
  grid={'logisticregression__C':C}
  vectorizer=CountVectorizer()
  clf=LogisticRegression(solver='lbfgs',max_iter=500,class_weight=class_weights)

  pipeline=make_pipeline(vectorizer,clf)
  grid_search=GridSearchCV(pipeline,param_grid=grid, cv=5, scoring='f1_macro')
  grid_search.fit(train_df.tweet, train_df.label)

  return grid_search.best_score_, grid_search.best_params_

In [14]:
cv_score,tuned_parameters=logistic_param_selection(train_df)

In [15]:
print('5 fold score',cv_score)
print('tuned parameter',tuned_parameters)

5 fold score 0.6963327961786592
tuned parameter {'logisticregression__C': 1}


#**Train and Test this Model**

In [16]:
tuned_parameters['logisticregression__C']

1

In [17]:
classifier=LogisticRegression(solver='lbfgs',C=tuned_parameters['logisticregression__C'],max_iter=500,class_weight=class_weights)
vectorizer=CountVectorizer()
train_vector=vectorizer.fit_transform(train_df.tweet)
test_vector=vectorizer.transform(test_df.tweet)

In [18]:
classifier.fit(train_vector,train_df.label)
y_pred=classifier.predict(test_vector)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(test_df.label,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82      1760
           1       0.65      0.60      0.62       888

    accuracy                           0.76      2648
   macro avg       0.73      0.72      0.72      2648
weighted avg       0.75      0.76      0.76      2648



#**Save the Model in the Pickle Format**

In [20]:
import pickle
import os
if not os.path.exists('models'):
  os.mkdir('models')
#save classifier
with open("models/logisticregression.pkl",'wb') as file:
  pickle.dump(classifier, file)
#save Vectorizer
with open("models/countvectorizer.pkl",'wb') as file:
  pickle.dump(vectorizer, file)
#save encoder
with open("models/labelencoder.pkl",'wb') as file:
  pickle.dump(enocder, file)

#**Inference**

load save pkl Files

In [21]:
import pickle
#Load the saved model
with open("models/logisticregression.pkl",'rb') as file:
  classifier=pickle.load(file)
#load the saved Vectrorizer
with open("models/countvectorizer.pkl",'rb') as file:
  vectorizer=pickle.load(file)
#load the saved encoder
with open("models/labelencoder.pkl",'rb') as file:
  encoder=pickle.load(file)

Preprocess

In [22]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
if not os.path.exists('nltkdata'):
  os.mkdir('nltkdata')
  nltk.download('wordnet',download_dir='nltkdata')
  nltk.download('stopwords',download_dir='nltkdata')
  nltk.download('omw-1.4',download_dir='nltkdata')
  nltk.download('punkt', download_dir='nltkdata')
nltk.data.path.append("nltkdata")

lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

In [23]:
import re
import string
def preprocessing(text):
  #apply preprocessing
  text=text.lower()
  text=re.sub('\d+','',text)
  text=re.sub('@user','',text)
  text=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]'
          '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
  text=text.translate(str.maketrans('','',string.punctuation))
  text=re.sub('[^\w\s]', '',text)

  text=[lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(text)]
  text=[item for item in text if item not in stop]
  text="  ".join(text)
  return text

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\w'
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\2497343809.py:6: SyntaxWarning: invalid escape sequence '\d'
  text=re.sub('\d+','',text)
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\2497343809.py:9: SyntaxWarning: invalid escape sequence '\('
  '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
C:\Users\hp\AppData\Local\Temp\ipykernel_12608\2497343809.py:11: SyntaxWarning: invalid escape sequence '\w'
  text=re.sub('[^\w\s]', '',text)


#**Predict**

In [24]:
def predict_result(text):
  text=preprocessing(text)
  vect=vectorizer.transform([text])
  pred=classifier.predict(vect)
  result=encoder.inverse_transform(pred)
  return result[0]

In [25]:
import nltk
nltk.download('punkt_tab', download_dir='nltkdata')

[nltk_data] Downloading package punkt_tab to nltkdata...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [26]:
text='Hello , will you shut your mouth !'
predict_result(text)

'OFF'

In [27]:
text='How are you ?'
predict_result(text)

'NOT'

#**Sys Argument**

In [28]:
%%writefile app.py
import sys
import pickle
#load the saved model

with open("models/logisticregression.pkl",'rb') as file:
  classifier=pickle.load(file)
#load the saved Vectrorizer
with open("models/countvectorizer.pkl",'rb') as file:
  vectorizer=pickle.load(file)
#load the saved encoder
with open("models/labelencoder.pkl",'rb') as file:
  encoder=pickle.load(file)

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
if not os.path.exists('nltkdata'):
  os.mkdir('nltkdata')
  nltk.download('wordnet',download_dir='nltkdata')
  nltk.download('stopwords',download_dir='nltkdata')
  nltk.download('omw-1.4',download_dir='nltkdata')
  nltk.download('punkt_tab', download_dir='nltkdata')

nltk.data.path.append("nltkdata")

lemmatizer=WordNetLemmatizer()
w_tokenizer=nltk.tokenize.WhitespaceTokenizer()
stop=stopwords.words('english')

import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def preprocessing(text):
    # Apply preprocessing
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'@user', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', '', text)

    text = [lemmatizer.lemmatize(y) for y in word_tokenize(text)]
    text = [item for item in text if item not in stop]
    text = " ".join(text)
    return text

def predict_result(text):
  text=preprocessing(text)
  vect=vectorizer.transform([text])
  pred=classifier.predict(vect)
  result=encoder.inverse_transform(pred)
  if result[0] == 'OFF':
    result[0] = 'Its an Offensive word'
  elif result[0] == 'NOT':
    result[0] = 'Its an Non Offensive word'
  return result[0]

text=sys.argv[1]
pred=predict_result(text)
print('prediction of "{}" is {}'.format(text,pred))

Writing app.py


In [29]:
! python app.py "This is bullshit!!"

prediction of "This is bullshit!!" is Its an Offensive word


In [30]:
! python app.py "How are you?"

prediction of "How are you?" is Its an Non Offensive word
